<br>
<br>
<h1 align="center">__Workflow for Lambda phage MinION data analysis __</h1>
<br>
<br>
<h3 align="center">By Maria Camila Isaza Álvarez</h3>
<h4 align="center">April 2018</h4>
<br>

## Aproach and metodology
We wil be using the following workflow to analyze subsets of MinION data files 
<br>
<br>
<img src="workflow.png">

## 1. Data collection
We are going to work with Oxford Nanopore sequencing data from the Lambda phage. For this assembly we are taking the .fast5 files from all the folders of the workapace. There are six foders with fast5 files and each folder has a size of approximatetly 4.00 GB on disk and it contains around 4,000 items.

## 2. Basecalling
We will be using albacore 1.1.1, which is available to download from the ONT community

We run with the -h option to get the usage instructions and the optional arguments

In [2]:
!read_fast5_basecaller.py -h

usage: read_fast5_basecaller.py [-h] [-l] [-v] [-i INPUT] -t WORKER_THREADS -s
                                SAVE_PATH [--resume [X]] [-f FLOWCELL]
                                [-k KIT] [--barcoding] [-c CONFIG]
                                [-d DATA_PATH] [-b] [-r]
                                [-n FILES_PER_BATCH_FOLDER] [-o OUTPUT_FORMAT]
                                [-q READS_PER_FASTQ_BATCH]
                                [--disable_filtering] [--disable_pings]

ONT Albacore Sequencing Pipeline Software

optional arguments:
  -h, --help            show this help message and exit
  -l, --list_workflows  List standard flowcell / kit combinations.
  -v, --version         Print the software version.
  -i INPUT, --input INPUT
                        Folder containing read fast5 files (if not present,
                        will expect file names on stdin).
  -t WORKER_THREADS, --worker_threads WORKER_THREADS
                        Number of worker threads to use.
  -s SA

Then we run the option -l to get the list of available configs used during the sequencing

In [3]:
!read_fast5_basecaller.py -l

Parsing config files in /Users/camilaisaza/anaconda3/lib/python3.6/site-packages/albacore/data_versioned.
Available flowcell + kit combinations are:
flowcell    kit         barcoding  config file
FLO-MIN106  SQK-DCS108             r94_450bps_linear.cfg
FLO-MIN106  SQK-LSK108             r94_450bps_linear.cfg
FLO-MIN106  SQK-LWB001  included   r94_450bps_linear.cfg
FLO-MIN106  SQK-LWP001             r94_450bps_linear.cfg
FLO-MIN106  SQK-PCS108             r94_450bps_linear.cfg
FLO-MIN106  SQK-PSB004  included   r94_450bps_linear.cfg
FLO-MIN106  SQK-PSK004             r94_450bps_linear.cfg
FLO-MIN106  SQK-RAB201  included   r94_450bps_linear.cfg
FLO-MIN106  SQK-RAB204  included   r94_450bps_linear.cfg
FLO-MIN106  SQK-RAD002             r94_450bps_linear.cfg
FLO-MIN106  SQK-RAD003             r94_450bps_linear.cfg
FLO-MIN106  SQK-RAD004             r94_450bps_linear.cfg
FLO-MIN106  SQK-RAS201             r94_450bps_linear.cfg
FLO-MIN106  SQK-RBK001  included   r94_450bps_linear.cfg
FLO-MI

Finally, we choose the arguments -i, -t, -s, -o and -c to do the base-calling and we select the config file r94_450bps_linear.cfg as the flowcell and the kit used were FLO-MIN106 and SQK-DCS108 respectively. The optional arguments means:
-  -i: input file 
-  -r: recursively search
-  -t: number of threads
-  -s: output directory
-  -o: output format
-  -c: config file

We cd into the directory containing the reads files and then we run the code: 

In [1]:
!read_fast5_basecaller.py -i ~/Documents/9._Noveno_semestre_2018-1/Analisis_de_secuencias/All_folders_taller_1/workspace -r -t 4 -s ~/Documents/9._Noveno_semestre_2018-1/Analisis_de_secuencias/All_folders_taller_1/outputalbacore -o fastq,fast5 -c r94_450bps_linear.cfg

| 23083 of 23083|###########################################|100% Time: 11:31:41


## 3 Extraction of FASTQ file using Poretools

#### 3.1 Extraction of FASTQ file using Poretools with fastq files
As poretools uses Python 2 we have to create a virtual environment to install and run the program

To run a virtualenv with python 2, we type:

In [9]:
!source activate python2

An then, we run poretools with the code:

In [10]:
!poretools fastq /Users/camilaisaza/Documents/9._Noveno_semestre_2018-1/Analisis_de_secuencias/All_folders_taller_1/outputalbacore/workspace/pass > /Users/camilaisaza/Documents/9._Noveno_semestre_2018-1/Analisis_de_secuencias/All_folders_taller_1/outputporetools.fq

#### 3.2 Extraction of FASTA file using poretools with fasta files

In [19]:
!source activate python2

In [18]:
!poretools fasta /Users/camilaisaza/Documents/9._Noveno_semestre_2018-1/Analisis_de_secuencias/All_folders_taller_1/outputalbacore/workspace/pass > /Users/camilaisaza/Documents/9._Noveno_semestre_2018-1/Analisis_de_secuencias/All_folders_taller_1/outputporetools.fasta

To obtain read lenght distribution we run:

In [2]:
#poretools hist /Users/camilaisaza/Documents/9._Noveno_semestre_2018-1/Analisis_de_secuencias/All_folders_taller_1/outputalbacore/workspace/pass

## 4. _De novo_ assembly

We are going to use Minimap2 and miniasm; Minimap for mapping and miniasm for assembly. These softwares are usefull  for long, noisy reads, they do not have a correction or consensus step, and therefore the resulting assemblies are contiguous (i.e. long) but very noisy.

We start the mapping with an all against all comparison:

#### 4.1 De novo assembly with fastq files

In [2]:
!minimap2 -x ava-ont /Users/camilaisaza/Documents/9._Noveno_semestre_2018-1/Analisis_de_secuencias/All_folders_taller_1/outputporetools.fq /Users/camilaisaza/Documents/9._Noveno_semestre_2018-1/Analisis_de_secuencias/All_folders_taller_1/outputporetools.fq > /Users/camilaisaza/Documents/9._Noveno_semestre_2018-1/Analisis_de_secuencias/All_folders_taller_1/outputminimap2.paf.gz

[M::mm_idx_gen::7.085*0.90] collected minimizers
[M::mm_idx_gen::8.024*1.14] sorted minimizers
[M::main::8.024*1.14] loaded/built the index for 14916 target sequence(s)
[M::mm_mapopt_update::8.199*1.14] mid_occ = 443
[M::mm_idx_stat] kmer size: 15; skip: 5; is_hpc: 0; #seq: 14916
[M::mm_idx_stat::8.307*1.14] distinct minimizers: 10917685 (78.98% are singletons); average occurrences: 2.313; average spacing: 2.948
[M::worker_pipeline::282.498*1.59] mapped 14916 sequences
[M::main] Version: 2.10-r761
[M::main] CMD: minimap2 -x ava-ont /Users/camilaisaza/Documents/9._Noveno_semestre_2018-1/Analisis_de_secuencias/All_folders_taller_1/outputporetools.fq /Users/camilaisaza/Documents/9._Noveno_semestre_2018-1/Analisis_de_secuencias/All_folders_taller_1/outputporetools.fq
[M::main] Real time: 282.970 sec; CPU: 448.180 sec


#### 4.2 De novo assembly with fastq files 

In [20]:
!minimap2 -x ava-ont /Users/camilaisaza/Documents/9._Noveno_semestre_2018-1/Analisis_de_secuencias/All_folders_taller_1/outputporetools.fasta /Users/camilaisaza/Documents/9._Noveno_semestre_2018-1/Analisis_de_secuencias/All_folders_taller_1/outputporetools.fasta > /Users/camilaisaza/Documents/9._Noveno_semestre_2018-1/Analisis_de_secuencias/All_folders_taller_1/outputminimap2defasta.paf

[M::mm_idx_gen::5.781*0.98] collected minimizers
[M::mm_idx_gen::6.697*1.25] sorted minimizers
[M::main::6.697*1.25] loaded/built the index for 14916 target sequence(s)
[M::mm_mapopt_update::6.877*1.24] mid_occ = 443
[M::mm_idx_stat] kmer size: 15; skip: 5; is_hpc: 0; #seq: 14916
[M::mm_idx_stat::6.986*1.24] distinct minimizers: 10917685 (78.98% are singletons); average occurrences: 2.313; average spacing: 2.948
[M::worker_pipeline::344.570*1.28] mapped 14916 sequences
[M::main] Version: 2.10-r761
[M::main] CMD: minimap2 -x ava-ont /Users/camilaisaza/Documents/9._Noveno_semestre_2018-1/Analisis_de_secuencias/All_folders_taller_1/outputporetools.fasta /Users/camilaisaza/Documents/9._Noveno_semestre_2018-1/Analisis_de_secuencias/All_folders_taller_1/outputporetools.fasta
[M::main] Real time: 344.932 sec; CPU: 441.753 sec


#### 4.1.2 Then we do the actual assembly typing:

In [4]:
!miniasm -f /Users/camilaisaza/Documents/9._Noveno_semestre_2018-1/Analisis_de_secuencias/All_folders_taller_1/outputporetools.fq /Users/camilaisaza/Documents/9._Noveno_semestre_2018-1/Analisis_de_secuencias/All_folders_taller_1/outputminimap2.paf.gz > /Users/camilaisaza/Documents/9._Noveno_semestre_2018-1/Analisis_de_secuencias/All_folders_taller_1/ensamble_raw.gfa

[M::main] ===> Step 1: reading read mappings <===
[M::ma_hit_read::65.960*0.32] read 13446574 hits; stored 15124789 hits and 9015 sequences (67250212 bp)
[M::main] ===> Step 2: 1-pass (crude) read selection <===
[M::ma_hit_sub::69.801*0.36] 8968 query sequences remain after sub
[M::ma_hit_cut::70.025*0.36] 14971747 hits remain after cut
[M::ma_hit_flt::70.300*0.36] 14664791 hits remain after filtering; crude coverage after filtering: 1165.79
[M::main] ===> Step 3: 2-pass (fine) read selection <===
[M::ma_hit_sub::71.333*0.37] 8956 query sequences remain after sub
[M::ma_hit_cut::71.536*0.37] 14657555 hits remain after cut
[M::ma_hit_contained::71.862*0.38] 12 sequences and 96 hits remain after containment removal
[M::main] ===> Step 4: graph cleaning <===
[M::ma_sg_gen] read 86 arcs
[M::main] ===> Step 4.1: transitive reduction <===
[M::asg_arc_del_trans] transitively reduced 57 arcs
[M::asg_arc_del_multi] removed 0 multi-arcs
[M::asg_arc_del_asymm] removed 1 asymmetric arcs
[M::main] 

#### 4.2.2 The actual assembly with the fasta format would be:

In [21]:
!miniasm -f /Users/camilaisaza/Documents/9._Noveno_semestre_2018-1/Analisis_de_secuencias/All_folders_taller_1/outputporetools.fasta /Users/camilaisaza/Documents/9._Noveno_semestre_2018-1/Analisis_de_secuencias/All_folders_taller_1/outputminimap2defasta.paf > /Users/camilaisaza/Documents/9._Noveno_semestre_2018-1/Analisis_de_secuencias/All_folders_taller_1/ensamble_rawdefasta.gfa

[M::main] ===> Step 1: reading read mappings <===
[M::ma_hit_read::45.639*0.46] read 13446574 hits; stored 15124789 hits and 9015 sequences (67250212 bp)
[M::main] ===> Step 2: 1-pass (crude) read selection <===
[M::ma_hit_sub::49.555*0.50] 8968 query sequences remain after sub
[M::ma_hit_cut::49.778*0.50] 14971747 hits remain after cut
[M::ma_hit_flt::50.045*0.51] 14664791 hits remain after filtering; crude coverage after filtering: 1165.79
[M::main] ===> Step 3: 2-pass (fine) read selection <===
[M::ma_hit_sub::51.106*0.52] 8956 query sequences remain after sub
[M::ma_hit_cut::51.313*0.52] 14657555 hits remain after cut
[M::ma_hit_contained::51.665*0.52] 12 sequences and 96 hits remain after containment removal
[M::main] ===> Step 4: graph cleaning <===
[M::ma_sg_gen] read 86 arcs
[M::main] ===> Step 4.1: transitive reduction <===
[M::asg_arc_del_trans] transitively reduced 57 arcs
[M::asg_arc_del_multi] removed 0 multi-arcs
[M::asg_arc_del_asymm] removed 1 asymmetric arcs
[M::main] 

We transform the format of output file from .gfa to .fasta

In [5]:
!awk '/^S/{print ">"$2"\n"$3}' /Users/camilaisaza/Documents/9._Noveno_semestre_2018-1/Analisis_de_secuencias/All_folders_taller_1/ensamble_raw.gfa | fold > /Users/camilaisaza/Documents/9._Noveno_semestre_2018-1/Analisis_de_secuencias/All_folders_taller_1/ensamble_raw.fasta

In [22]:
!awk '/^S/{print ">"$2"\n"$3}' /Users/camilaisaza/Documents/9._Noveno_semestre_2018-1/Analisis_de_secuencias/All_folders_taller_1/ensamble_rawdefasta.gfa | fold > /Users/camilaisaza/Documents/9._Noveno_semestre_2018-1/Analisis_de_secuencias/All_folders_taller_1/ensamble_rawdefasta.fasta

Finally, we count the number of contig by typing:

In [6]:
!grep -c ">" /Users/camilaisaza/Documents/9._Noveno_semestre_2018-1/Analisis_de_secuencias/All_folders_taller_1/ensamble_raw.fasta

1


In [23]:
!grep -c ">" /Users/camilaisaza/Documents/9._Noveno_semestre_2018-1/Analisis_de_secuencias/All_folders_taller_1/ensamble_rawdefasta.fasta

1


## 5. Assembly statistics

We can run assembly-stats to conduct basic statistics like N50, assembly size and number of contigs

In [16]:
!assembly-stats /Users/camilaisaza/Documents/9._Noveno_semestre_2018-1/Analisis_de_secuencias/All_folders_taller_1/ensamble_raw.fasta

stats for /Users/camilaisaza/Documents/9._Noveno_semestre_2018-1/Analisis_de_secuencias/All_folders_taller_1/ensamble_raw.fasta
sum = 46890, n = 1, ave = 46890.00, largest = 46890
N50 = 46890, n = 1
N60 = 46890, n = 1
N70 = 46890, n = 1
N80 = 46890, n = 1
N90 = 46890, n = 1
N100 = 46890, n = 1
N_count = 0
Gaps = 0


We can also run infoseq to know the length and %GC of the contigs:

In [1]:
!infoseq -auto -only -accession -length -pgc /Users/camilaisaza/Documents/9._Noveno_semestre_2018-1/Analisis_de_secuencias/All_folders_taller_1/ensamble_rawdefasta.fasta

Accession      Length %GC    
-              46890  50.15  


In [11]:
%matplotlib inline

import matplotlib
matplotlib.use("Agg")
import numpy as np
import pandas as pd

import warnings
import matplotlib.cbook
warnings.filterwarnings("ignore",category=matplotlib.cbook.mplDeprecation)

import matplotlib.pyplot as plt

import ast
import matplotlib
matplotlib.use('Agg')
import matplotlib.pyplot as plt
import matplotlib.patches as mplpatches
from matplotlib.colors import LinearSegmentedColormap
import numpy as np
import pandas as pd
import os.path as opath
from sys import stderr
from pauvre.functions import parse_fastq_length_meanqual, print_images, filter_fastq_length_meanqual
from pauvre.stats import stats
import pauvre.rcparams as rc
import logging

/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:4: UserWarning: 
This call to matplotlib.use() has no effect because the backend has already
been chosen; matplotlib.use() must be called *before* pylab, matplotlib.pyplot,
or matplotlib.backends is imported for the first time.

The backend was *originally* set to 'module://ipykernel.pylab.backend_inline' by the following code:
  File "/anaconda3/lib/python3.6/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/anaconda3/lib/python3.6/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/anaconda3/lib/python3.6/site-packages/traitlets/config/application.py", line 658, in launch_instance
    app.start()
  File "/anaconda3/lib/python3.6/site-packages/ipykernel/kernelapp.py", line 478, in start
    self.io_loop.start()
  File "/anaconda3/lib/python3.6/site-packages/z

ModuleNotFoundError: No module named 'pauvre'

In [14]:
!pauvre marginplot --fastq /Users/camilaisaza/Documents/9._Noveno_semestre_2018-1/Analisis_de_secuencias/All_folders_taller_1/outputporetools.fq

Keeping reads that satisfy: length >= 0 and meanQual >= 0
# Fastq stats for outputporetools.fq, reads >= 0bp
numReads: 14916
%totalNumReads: 100.00
numBasepairs: 74455398
%totalBasepairs: 100.00
meanLen: 4991.6464199517295
medianLen: 2752.5
minLen: 156
maxLen: 108424
N50: 8415
L50: 2581

# Fastq stats for outputporetools.fq, reads >= 1000bp
numReads: 12601
%totalNumReads: 84.48
numBasepairs: 72848884
%totalBasepairs: 97.84
meanLen: 5781.198635028966
medianLen: 4146.0
minLen: 1000
maxLen: 108424
N50: 8619
L50: 2487

# Fastq stats for outputporetools.fq, reads >= 5000bp
numReads: 5239
%totalNumReads: 35.12
numBasepairs: 54469188
%totalBasepairs: 73.16
meanLen: 10396.867341095629
medianLen: 8349.0
minLen: 5000
maxLen: 108424
N50: 11208
L50: 1549

# Fastq stats for outputporetools.fq, reads >= 10000bp
numReads: 1931
%totalNumReads: 12.95
numBasepairs: 31273271
%totalBasepairs: 42.00
meanLen: 16195.375970999483
medianLen: 13825.0
minLen: 10000
maxLen: 108424
N50: 16173
L50: 673


          

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/matplotlib/cbook/deprecation.py:107: MatplotlibDeprecationWarning: Passing one of 'on', 'true', 'off', 'false' as a boolean is deprecated; use an actual boolean (True/False) instead.
  warnings.warn(message, mplDeprecation, stacklevel=1)
plotting in the following window:
        0 <= Q-score (x-axis) <= 11.0
        0 <= length  (y-axis) <= 26416


Al hacer el ensamble con minimap2/miniasm de los fastq producidos por poretools se obtuvo un único contig que cubre todo el genoma del fago lambda, es por esto que todos los valores de la medida Nx son iguales. La longitud del contig fue de 46,890 bp, valor muy cercano al registrado en el NCBI de 48,502 bp. El contig obtenido tiene un contenido Guanina-citocina del 50.15% lo cual significa que este genoma cuenta con un alto grado de estabilidad. 